In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

### 교차검증의 종류
- 훈련용과 검증용으로 교차하여 모델학습을 시키는 방법으로 과적합을 줄이는데 사용한다
- KFold, SKFold(계층적 KFold) 이용
- cross_val_score, cross_validate를 이용하면 회귀에는 Kfold, 분류에는 SKFold가 사용된다
- cross_val_score(사용할 모델, x값, y값, scoring(평가할 스코어링 방법), cv 몇번 교차 검증할 것인가? n_jobs(cpu사용에 대한 지정 -1), verbose(진행상황보기), 교차 검증 되는 상황 체크, fit_params 등)

In [6]:
iris_data = load_iris()
data = iris_data.data
label = iris_data.target
dt_clf = DecisionTreeClassifier(random_state=111)
crs = cross_validate(dt_clf, data, label, scoring='accuracy', cv=5, return_train_score=True)

import pandas as pd
pd.DataFrame(crs)

,fit_time,score_time,test_score,train_score
0,0.000000,0.0,0.966667,1.0
1,0.000000,0.0,0.966667,1.0
2,0.001312,0.0,0.900000,1.0
3,0.000000,0.0,0.966667,1.0
4,0.007997,0.0,1.000000,1.0


In [6]:
#교차검증 진행
scores=cross_val_score(dt_clf, data,label, scoring='accuracy',cv=5)
print(np.round(np.mean(scores),3)) #교차검증의 평균

0.96


In [7]:
scores

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

In [17]:
#### Confusion matrix
from sklearn.metrics import make_scorer, precision_score
scores = cross_val_score(dt_clf, data, label, scoring='precision', cv=5)

C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\sklearn\model_selection\_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\sklearn\metrics\_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\sklearn\metrics\_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\sklearn\metrics\_classification.py", line 1573, in precision_recall_fscore_support
    labels = _check_set_wise_labels(y_true, y_pred, ave

  - 흔히 말하는 계산방식 정확도, 정밀도, 재현율, f1스코어, auc, roc 커브 등등 0,1 이진 분류에만 디폴트로 적용된 것
  - 이진분류는 문제없이 값들이 나옴
- 다중분류는 값을 평가하는 방식이 다름

In [19]:
scores = cross_val_score(dt_clf, data, label, scoring='precision_micro', cv=5)

### scoring 종류
- **Classification**
  - accuracy
  - f1
  - average_precision
  - precision etc
  - recall etc
  - roc etc
- **Clutstering**
  - mutual_info_score
  - rand_score
  - measure_score
- **Regression**
  - absolute_error
  - squared_error
  - poisson_deviance
  - gamma_deviance
  

## 다중분류 진행 시 정확도 외 다른 평가지표에 대한 가중치 설정 방법
- macro : 모든 예측 결과에 대해서 평균을 내어서 값을 계산하는 방법
- micro : 각각의 정답에 대한 개수를 가지고 나눠서 평균값을 구한다
- weighted : 가중 평균값을 구한다. 가중치를 주고 싶은 것에 줄 수 있다

- TP : 참긍정. 긍정인것 중 긍정이라고 예측
- FP : 거짓긍정. 부정인것 중 긍정이라고 예측
- FN : 거짓부정. 긍정인것 중 부정이라고 예측
- TN : 참부정. 부정인것 중 부정이라고 예측 

- **정확도**(예측이 현실에 부합할 확률) = $\frac{TP+TN}{TP+TN+FP+FN}$
- **정밀도**(긍정예측 중에 실제 긍정일 확률) = $\frac{TP}{TP+FP}$
- **민감도**(=재현율. 실제 긍정중 긍정이라고 예측할 확률) = $\frac{TP}{TP+FN}$
- **특이도**(실제로 부정일 때 예측도 부정일 확률) = $\frac{TN}{TN+FP}$
- **F1값** = $\frac{2*Precicion*Recall}{Precision+Recall}$

<img src="https://www.mariakhalusova.com/images/metrics/micro-macro-table.png">

$precision = \frac{TP_{total}}{TP_{total}+FP_{total}}$

$Macro-Precision = \frac{1}{3}(Precision_{bird}+Precision_{cat}+Precision_{dog})
= \frac{1}{3}(1+0.8+0.667)$

$Micro-Precision = \frac{TP_{total}}{TP_{total}+FP_{total}}=\frac{7}{7+2}$

$weighted-Precision = \frac {Precision_{bird}*N_{bird}+Precision_{cat}*N_{cat}+Precision_{dog}*N_{dog}}{Total number of samples} = \frac {1*2+0.8*4+0.667*3}{9}$

In [20]:
scores

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

## leaveOneOut 교차검증
-  단 하나의 관측값을 validation set으로 사용하고 나머지 n-1개 관측값은 train set으로 사용

## Group 교차검증
- 그룹핑은 그룹핑 데이터를 가지고 교차검증할 때 더 좋은 성능을 보여줄 수 있다
- 단순하게 교차검증을 cross 하는 게 아니라
- 데이터의 특성을 보고 내가 분석 할 것이 어떤 클래스별의 수치를 비교하는 것

In [8]:
import seaborn as sns
df = sns.load_dataset('titanic')
df_tt=df[['pclass','fare']]

In [9]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(dt_clf, df_tt, df['survived'],scoring='accuracy',cv=loo)

In [10]:
print(len(scores))
print(scores.mean())

891
0.6745230078563412


In [11]:
## cross val 응용하기
model = [DecisionTreeClassifier(),RandomForestClassifier(),LogisticRegression()]
name = ['DT','RF',"LR"]

for model,name in zip(model, name):
    print('*************사용한 알고리즘', name, ' *****************')
    for score in ['accuracy','precision','recall','f1']:
                  print(score)
                  print("--")
                  print(cross_val_score(model, df_tt, df['survived'], scoring=score, cv=10))


*************사용한 알고리즘 DT  *****************
accuracy
--
[0.67777778 0.61797753 0.64044944 0.66292135 0.6741573  0.71910112
 0.61797753 0.78651685 0.71910112 0.75280899]
precision
--
[0.63636364 0.5        0.54545455 0.59090909 0.6        0.63636364
 0.5        0.74193548 0.65517241 0.76      ]
recall
--
[0.4        0.35294118 0.35294118 0.38235294 0.44117647 0.61764706
 0.32352941 0.67647059 0.55882353 0.54285714]
f1
--
[0.49122807 0.4137931  0.42857143 0.46428571 0.50847458 0.62686567
 0.39285714 0.70769231 0.6031746  0.63333333]
*************사용한 알고리즘 RF  *****************
accuracy
--
[0.7        0.64044944 0.6741573  0.6741573  0.69662921 0.70786517
 0.68539326 0.78651685 0.71910112 0.7752809 ]
precision
--
[0.65384615 0.48148148 0.60714286 0.59259259 0.62962963 0.62857143
 0.66666667 0.6969697  0.67741935 0.74074074]
recall
--
[0.45714286 0.35294118 0.47058824 0.44117647 0.5        0.67647059
 0.38235294 0.70588235 0.61764706 0.62857143]
f1
--
[0.56666667 0.42622951 0.50847458 0.562